### Import libraries

In [10]:
import os
import pandas as pd
import pyodbc, struct
from azure import identity
from faker import Faker
from typing import Union
from fastapi import FastAPI
from pydantic import BaseModel
from sqlalchemy import create_engine

from dotenv import load_dotenv
load_dotenv(override=True)
connection_string = os.environ["AZURE_SQL_CONNECTIONSTRING"]

SQL_SERVER_USERNAME = os.environ["SQL_SERVER_USERNAME"]
SQL_SERVER_ENDPOINT = os.environ["SQL_SERVER_ENDPOINT"]
SQL_SERVER_PASSWORD = os.environ["SQL_SERVER_PASSWORD"]
SQL_SERVER_DATABASE = os.environ["SQL_SERVER_DATABASE"] 
driver = "{ODBC Driver 18 for SQL Server}"
sqlalchemy_driver = "ODBC Driver 18 for SQL Server"

## Creatng a connection to the SQL Server
conn = pyodbc.connect(f'DRIVER={driver};SERVER={SQL_SERVER_ENDPOINT};PORT=1433;DATABASE={SQL_SERVER_DATABASE};UID={SQL_SERVER_USERNAME};PWD={SQL_SERVER_PASSWORD}')  
cursor = conn.cursor()

from openai import AzureOpenAI
aoai_client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-07-01-preview"
)

#https://github.com/Azure-Samples/openai/blob/main/End_to_end_Solutions/AOAISearchDemo/scripts/prepopulate/populate_sql.py

#### Reading input files

In [11]:
relative_path = "../../../../data/processed/files/"
files = os.listdir(relative_path)

df = pd.concat([pd.read_parquet(relative_path+file) for file in files]).reset_index(drop=True)
df['title_vector'] = df['title_vector'].apply(lambda x: x.tolist())
df['content_vector'] = df['content_vector'].apply(lambda x: x.tolist())
df['id'] = df['chunk_id']

CONTAINER_ID = df['preprocessing_pipeline'][0]

#### Save CSV files as tables

#### Create other SQL Tables

In [12]:
cursor.execute("""
CREATE TABLE Customers (
  cust_id INTEGER,
  cust_name VARCHAR(1000),
  cust_email VARCHAR(1000),
  cust_phone VARCHAR(1000),
  cust_address VARCHAR(1000),
  PRIMARY KEY (cust_id)
);

CREATE TABLE Products (
  prod_id INTEGER,
  prod_name VARCHAR(1000),
  price FLOAT,
  category VARCHAR(1000),
  PRIMARY KEY (prod_id)
);

CREATE TABLE Merchants (
  merchant_id INTEGER,
  merchant_name VARCHAR(1000),
  merchant_region VARCHAR(1000),
  merchant_address VARCHAR(1000),
  PRIMARY KEY (merchant_id)
);

CREATE TABLE Stock (
  prod_id INTEGER,
  merchant_id INTEGER,
  stock INTEGER,
  PRIMARY KEY (prod_id, merchant_id),
  FOREIGN KEY (merchant_id) REFERENCES Merchants(merchant_id),
  FOREIGN KEY (prod_id) REFERENCES Products(prod_id)
);

CREATE TABLE Sales (
    sale_id INTEGER,
    cust_id INTEGER,
    merchant_id INTEGER,
    date DATETIME,
    total_price FLOAT,
    PRIMARY KEY (sale_id),
    FOREIGN KEY (cust_id) REFERENCES Customers(cust_id),
    FOREIGN KEY (merchant_id) REFERENCES Merchants(merchant_id)
);

CREATE TABLE Sales_Detail (
  sales_id INTEGER,
  prod_id INTEGER,
  quantity INTEGER,
  PRIMARY KEY (sales_id, prod_id),
  FOREIGN KEY (sales_id) REFERENCES Sales(sale_id),
  FOREIGN KEY (prod_id) REFERENCES Products(prod_id)
);
""")

cursor.commit()

In [13]:
import pandas as pd

In [14]:
relative_path = "../../../../data/processed/structured/"
files = os.listdir(relative_path)

In [15]:
files

['customers.csv',
 'greenTaxiData.csv',
 'merchants.csv',
 'products.csv',
 'sales.csv',
 'sales_detail.csv',
 'stock.csv',
 'yellowTaxiData.csv']

In [16]:
pd.read_csv(relative_path+"greenTaxiData.csv").head()

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,fareAmount,extra,mtaTax,improvementSurcharge,tipAmount,tollsAmount,ehailFee,totalAmount,tripType,transactionID
0,2,1/3/2016 21:02,1/3/2016 21:05,1,0.83,NaN,NaN,-73.987267,40.693836,-73.976112,...,4.5,0.5,0.5,0.3,0.00,0.0,NaN,5.80,1,JLYOPGED
1,2,1/19/2016 21:49,1/19/2016 21:54,1,1.27,NaN,NaN,-73.948456,40.801468,-73.959755,...,6.0,0.5,0.5,0.3,1.00,0.0,NaN,8.30,1,MRVEIORX
2,2,1/5/2016 9:46,1/5/2016 9:57,1,1.80,NaN,NaN,-73.955444,40.679798,-73.980309,...,9.5,0.0,0.5,0.3,2.06,0.0,NaN,12.36,1,TUQUVPZD
3,1,1/8/2016 17:49,1/8/2016 17:52,1,0.50,NaN,NaN,-73.922935,40.760818,-73.925499,...,4.0,1.0,0.5,0.3,1.15,0.0,NaN,6.95,1,HDXVLQUX
4,1,1/29/2016 10:28,1/29/2016 10:34,1,0.90,NaN,NaN,-73.923042,40.664940,-73.911041,...,6.0,0.0,0.5,0.3,0.00,0.0,NaN,6.80,1,KMKOEINU


In [17]:
pd.read_csv(relative_path+"yellowTaxiData.csv").head()

,vendorID,tpepPickupDateTime,tpepDropoffDateTime,passengerCount,tripDistance,puLocationId,doLocationId,startLon,startLat,endLon,...,storeAndFwdFlag,paymentType,fareAmount,extra,mtaTax,improvementSurcharge,tipAmount,tollsAmount,totalAmount,transactionID
0,2,1/6/2016 12:09,1/6/2016 12:22,1,2.09,NaN,NaN,-73.982071,40.746059,-74.004623,...,N,1,10.5,0.0,0.5,0.3,2.26,0.0,13.56,IWZHYXRG
1,1,1/3/2016 17:57,1/3/2016 18:08,3,1.50,NaN,NaN,-73.966270,40.764835,-73.984550,...,N,2,8.5,1.0,0.5,0.3,0.00,0.0,10.30,KWHGBWXW
2,1,1/18/2016 7:37,1/18/2016 7:47,1,1.80,NaN,NaN,0.000000,0.000000,0.000000,...,N,1,8.5,0.0,0.5,0.3,1.85,0.0,11.15,WYUVOPUP
3,2,1/26/2016 0:31,1/26/2016 0:38,1,1.96,NaN,NaN,-73.990623,40.755360,-73.978958,...,N,1,8.0,0.5,0.5,0.3,1.00,0.0,10.30,REQCUWHO
4,2,1/20/2016 23:37,1/20/2016 23:51,1,3.60,NaN,NaN,-73.985283,40.760262,-74.011276,...,N,1,13.5,0.5,0.5,0.3,2.50,0.0,17.30,ELSOVLCM


In [18]:
pd.read_csv(relative_path+'yellowTaxiData.csv').columns

Index(['vendorID', 'tpepPickupDateTime', 'tpepDropoffDateTime',
       'passengerCount', 'tripDistance', 'puLocationId', 'doLocationId',
       'startLon', 'startLat', 'endLon', 'endLat', 'rateCodeId',
       'storeAndFwdFlag', 'paymentType', 'fareAmount', 'extra', 'mtaTax',
       'improvementSurcharge', 'tipAmount', 'tollsAmount', 'totalAmount',
       'transactionID'],
      dtype='object')

In [19]:
pd.read_csv(relative_path+'greenTaxiData.csv').columns

Index(['vendorID', 'lpepPickupDatetime', 'lpepDropoffDatetime',
       'passengerCount', 'tripDistance', 'puLocationId', 'doLocationId',
       'pickupLongitude', 'pickupLatitude', 'dropoffLongitude',
       'dropoffLatitude', 'rateCodeID', 'storeAndFwdFlag', 'paymentType',
       'fareAmount', 'extra', 'mtaTax', 'improvementSurcharge', 'tipAmount',
       'tollsAmount', 'ehailFee', 'totalAmount', 'tripType', 'transactionID'],
      dtype='object')

#### Inserting Tables

In [20]:
import numpy as np

In [21]:
def CSVToSQL(table_name,file_location):
    # Read the CSV file
    df = pd.read_csv(file_location).fillna(np.nan).replace([np.nan], [None])
    for index, row in df.iterrows():
        insert_query = f"""
        INSERT INTO {table_name} ({', '.join(df.columns)})
        VALUES ({', '.join(['?' for _ in df.columns])})
        """
        cursor.execute(insert_query, tuple(row))
    conn.commit()

In [22]:
relative_path = "../../../../data/processed/structured/"
files = os.listdir(relative_path)

CSVToSQL("Customers",relative_path+'/customers.csv')
CSVToSQL("Products",relative_path+'/products.csv')
CSVToSQL("Merchants",relative_path+'/merchants.csv')
CSVToSQL("Stock",relative_path+'/stock.csv')
CSVToSQL("Sales",relative_path+'/sales.csv')
CSVToSQL("Sales_Detail",relative_path+'/sales_detail.csv')

cursor.commit()

### Generating description this data

In [23]:
string_to_description = ""
for file in files:
    table_name = file.replace(".csv","")
    values = str(pd.read_csv(relative_path+file).head().to_dict(orient='records'))
    sample_string = "Table name: " + table_name + " \n" + "Values: " + values+ " \n"
    string_to_description += sample_string

In [24]:
print(string_to_description)

Table name: customers 
Values: [{'cust_id': 1, 'cust_name': 'John Doe', 'cust_email': 'john.doe@example.com', 'cust_phone': '555-123-4567', 'cust_address': '123 Main St, City A'}, {'cust_id': 2, 'cust_name': 'Jane Smith', 'cust_email': 'jane.smith@example.com', 'cust_phone': '555-987-6543', 'cust_address': '456 High St, City B'}, {'cust_id': 3, 'cust_name': 'Michael Johnson', 'cust_email': 'michael.johnson@example.com', 'cust_phone': '555-456-7890', 'cust_address': '789 First St, City C'}, {'cust_id': 4, 'cust_name': 'Sara Williams', 'cust_email': 'sara.williams@example.com', 'cust_phone': '555-321-6549', 'cust_address': '147 Second St, City D'}, {'cust_id': 5, 'cust_name': 'Kevin Brown', 'cust_email': 'kevin.brown@example.com', 'cust_phone': '555-987-3214', 'cust_address': '258 Third St, City E'}] 
Table name: greenTaxiData 
Values: [{'vendorID': 2, 'lpepPickupDatetime': '1/3/2016 21:02', 'lpepDropoffDatetime': '1/3/2016 21:05', 'passengerCount': 1, 'tripDistance': 0.83, 'puLocationId

In [25]:
def GetToolDescription(string_to_description):
    messages = [{"role":"system","content":"You are an agent that takes information about tables, including table names and sample values, with the objective of generating a description that will be used to select an OpenAI tool that will help users to interact with the mentioned tables. Please provide examples of each field"},
            {"role":"user", "content":"Information about tables "+ string_to_description}]

    response = aoai_client.chat.completions.create(model="gpt-4.1",  
                                    messages = messages, 
                                    temperature=0.2,  
                                    max_tokens=2000,
                                    seed = 42)
    DBDescription = response.choices[0].message.content
    print(DBDescription)

    return DBDescription

In [26]:
tool_description = GetToolDescription(string_to_description)

Here is a structured description of the tables and their sample values, suitable for selecting an OpenAI tool for interaction:

---

**Table Descriptions for Tool Selection**

1. Table Name: customers  
   - Description: Contains customer information including unique ID, name, email, phone number, and address.
   - Sample Fields:  
     - cust_id: 1  
     - cust_name: "John Doe"  
     - cust_email: "john.doe@example.com"  
     - cust_phone: "555-123-4567"  
     - cust_address: "123 Main St, City A"

2. Table Name: greenTaxiData  
   - Description: Stores detailed records of green taxi trips, including vendor, pickup/dropoff times and locations, trip distance, fare breakdown, payment type, and transaction ID.
   - Sample Fields:  
     - vendorID: 2  
     - lpepPickupDatetime: "1/3/2016 21:02"  
     - lpepDropoffDatetime: "1/3/2016 21:05"  
     - passengerCount: 1  
     - tripDistance: 0.83  
     - pickupLongitude: -73.98726654  
     - pickupLatitude: 40.69383621  
     - fare

#### Saving description to file

In [27]:
with open("../../../../data/processed/tool_description/GetSQLDBAnswerDescription.txt", "w") as file:
    # Write the string to the file
    file.write(tool_description)

In [28]:
result = cursor.execute("SELECT prod_name, category, price FROM Products WHERE prod_name like '%Surface%' and category like '%accessory%';")
[row for row in result.fetchall()]

[('Surface Slim Pen 2', 'Accessory', 94.99),
 ('Surface Pen', 'Accessory', 99.99),
 ('Surface Dock', 'Accessory', 199.99)]

In [29]:
result = cursor.execute("SELECT SUM(stock) AS total_stock FROM Stock WHERE prod_id IN (SELECT prod_id FROM Products WHERE prod_name LIKE '%Surface Laptop 5%');")
[row for row in result.fetchall()]

[(2527,)]

In [30]:
cursor.execute("""
CREATE TABLE greenTaxiData (  
    vendorID INT,  
    lpepPickupDatetime VARCHAR(400),  
    lpepDropoffDatetime VARCHAR(400),  
    passengerCount INT,  
    tripDistance FLOAT,  
    puLocationId FLOAT,
    doLocationId FLOAT,
    pickupLongitude FLOAT,  
    pickupLatitude FLOAT,  
    dropoffLongitude FLOAT,  
    dropoffLatitude FLOAT,  
    rateCodeID INT,  
    storeAndFwdFlag VARCHAR(400),  
    paymentType INT,  
    fareAmount FLOAT,  
    extra FLOAT,  
    mtaTax FLOAT,  
    improvementSurcharge FLOAT,  
    tipAmount FLOAT,  
    tollsAmount FLOAT,  
    ehailFee FLOAT,
    totalAmount FLOAT,  
    tripType FLOAT,
    transactionID VARCHAR(100), 
    PRIMARY KEY (transactionID),  
); 

CREATE TABLE yellowTaxiData (  
    vendorID INT,  
    tpepPickupDateTime VARCHAR(400),  
    tpepDropoffDateTime VARCHAR(400),  
    passengerCount INT,  
    tripDistance FLOAT,  
    puLocationId FLOAT,  
    doLocationId FLOAT,  
    startLon FLOAT,  
    startLat FLOAT,  
    endLon FLOAT,  
    endLat FLOAT,  
    rateCodeId INT,  
    storeAndFwdFlag VARCHAR(400),  
    paymentType INT,  
    fareAmount FLOAT,  
    extra FLOAT,  
    mtaTax FLOAT,  
    improvementSurcharge FLOAT,  
    tipAmount FLOAT,  
    tollsAmount FLOAT,  
    totalAmount FLOAT,
    transactionID VARCHAR(100), 
    PRIMARY KEY (transactionID),  
);   
""")

cursor.commit()

In [31]:
def CSVToSQL(table_name,file_location):
    # Read the CSV file
    df = pd.read_csv(file_location).fillna(np.nan).replace([np.nan], [None]).iloc[100:]
    for index, row in df.iterrows():
        insert_query = f"""
        INSERT INTO {table_name} ({', '.join(df.columns)})
        VALUES ({', '.join(['?' for _ in df.columns])})
        """
        cursor.execute(insert_query, tuple(row))
    conn.commit()

In [32]:
relative_path = "../../../../data/processed/structured/"
files = os.listdir(relative_path)


CSVToSQL("greenTaxiData",relative_path+'/greenTaxiData.csv')
CSVToSQL("yellowTaxiData",relative_path+'/yellowTaxiData.csv')

cursor.commit()